<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

In [ ]:
# pip install openeo==0.23.0

# 2.3 Data Access and Basic Processing

## Reduce Operators

When computing statistics over time or indices based on multiple bands, it is possible to use reduce operators.

In openEO we can use the [reduce_dimension](https://processes.openeo.org/#reduce_dimension) process, which applies a reducer to a data cube dimension by collapsing all the values along the specified dimension into an output value computed by the reducer.

Reduce the temporal dimension to a single value, the mean for instance:

In [ ]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}
temporal_extent = ["2022-05-10T00:00:00Z","2022-06-30T00:00:00Z"]
bands = ["red","nir"]
datacube = local_conn.load_stac(url=url,
                                spatial_extent=spatial_extent,
                                temporal_extent=temporal_extent,
                                bands=bands)

datacube_min_time = datacube.reduce_dimension(dimension="time",reducer="min")
datacube_min_time

Check what happens to the datacube inspecting the resulting xArray object:

In [ ]:
datacube_min_time.execute()

It is possible to reduce in the same way all the available dimensions of the datacube.

We can, for instance, reduce the band dimension similarly as we did for the temporal dimension:

In [ ]:
datacube_mean_band = datacube.reduce_dimension(dimension="band",reducer="mean")

The result will now contain values resulting from the average of the bands:

In [ ]:
datacube_mean_band.execute()

**Quiz hint: look carefully at number of pixels of the loaded datacube!**

The reducer could be again a single process, but when computing spectral indices like NDVI, NDSI etc. an arithmentical formula is used instead.

For instance, the [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) formula can be expressed using a `reduce_dimension` process over the `bands` dimension:

$$ NDVI = {{NIR - RED} \over {NIR + RED}} $$

In [ ]:
def NDVI(data):
    red = data.array_element(index=0)
    nir = data.array_element(index=1)
    ndvi = (nir - red)/(nir + red)
    return ndvi

ndvi = datacube.reduce_dimension(reducer=NDVI,dimension="band")
ndvi.execute()

Additionally, it is possible to reduce both spatial dimensions of the datacube at the same time.

To do this, we need the `reduce_spatial` process.

We show an example where it is used to compute the standard deviation `sd`.

In [ ]:
datacube_spatial_sd = datacube.reduce_spatial(reducer="sd")
datacube_spatial_sd

Verify that the spatial dimensions were collapsed:

In [ ]:
datacube_spatial_sd.execute()